## Shiny App

This notebook will be for setting up Shiny app code for map deployment

In [24]:
# import packages
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
from shiny import App, reactive, render, ui

# import functions
# import sys
# sys.path.append("..")
# from src import *
import src.shiny_functions as sf

# visulaztion packages
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [25]:
# import gdf
gdf = gpd.GeoDataFrame(pd.read_pickle("./data/pickle/gdf_dc_filter"))

In [26]:
# preview
gdf.head()

,state,population,state_fips,pop_per_1k,count_per_1k,injured_per_1k,killed_per_1k,total_per_1k,statens,affgeoid,stusps,lsad,aland,awater,geometry,lawtotal
0,Illinois,12716164.0,17,12716.0,1.68,7.57,1.20,8.78,01779784,0400000US17,IL,00,143778561906,6216493488,"POLYGON ((-91.51297 40.18106, -91.51107 40.188...",70.0
1,Texas,28635442.0,48,28635.0,0.51,2.04,0.70,2.74,01779801,0400000US48,TX,00,676681550479,18978390713,"MULTIPOLYGON (((-94.71830 29.72885, -94.71721 ...",18.0
2,California,39346023.0,06,39346.0,0.37,1.42,0.45,1.88,01779778,0400000US06,CA,00,403671756816,20293573058,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",111.0
3,New York,19514849.0,36,19515.0,0.53,2.33,0.29,2.62,01779796,0400000US36,NY,00,122049142543,19257128505,"MULTIPOLYGON (((-72.03683 41.24984, -72.03496 ...",77.0
4,Pennsylvania,12794885.0,42,12795.0,0.81,3.38,0.52,3.89,01779798,0400000US42,PA,00,115881784866,3397909903,"POLYGON ((-80.51989 40.90666, -80.51963 40.911...",39.0


In [27]:
from platform import python_version
python_version()

'3.9.13'

In [28]:
    sf.interactive_choro(gdf,col_key_val_list=["state","count_per_1k"],
                 tooltip_col_list=["stusps","count_per_1k","pop_per_1k"],
                 tooltip_name_list=["State","Mass Shooting per 1k","Population  per 1k"],
                 popup_col_list=["state","population","lawtotal"], popup_name_list=["State","Population",
                                                                                    "Total Gun Laws"])

NameError: name 'folium' is not defined

In [ ]:
app_ui = ui.page_fluid(
    ui.div(
#         ui.input_slider("zoom", "Map zoom level", value=12, min=1, max=18),
#         ui.output_ui("map_bounds"),
        style=css(
            display="flex", justify_content="center", align_items="center", gap="2rem"
        ),
    ),
    output_widget("map"),
)


def server(input, output, session):
    # Initialize and display when the session starts (1)
    # plot count per 1k
    sf.interactive_choro(gdf,col_key_val_list=["state","count_per_1k"],
                 tooltip_col_list=["stusps","count_per_1k","pop_per_1k"],
                 tooltip_name_list=["State","Mass Shooting per 1k","Population  per 1k"],
                 popup_col_list=["state","population","lawtotal"], popup_name_list=["State","Population",
                                                                                    "Total Gun Laws"])
    register_widget("map", map)

    # When the slider changes, update the map's zoom attribute (2)
    @reactive.Effect
    def _():
        map.zoom = input.zoom()

    # When zooming directly on the map, update the slider's value (2 and 3)
    @reactive.Effect
    def _():
        ui.update_slider("zoom", value=reactive_read(map, "zoom"))

    # Everytime the map's bounds change, update the output message (3)
    @output
    @render.ui
    def map_bounds():
        center = reactive_read(map, "center")
        if len(center) == 0:
            return

        lat = round(center[0], 4)
        lon = (center[1] + 180) % 360 - 180
        lon = round(lon, 4)

        return ui.p(f"Latitude: {lat}", ui.br(), f"Longitude: {lon}")


app = App(app_ui, server)
